In [184]:
import pandas as pd
import numpy as np
import os
import collections
import pickle
import datetime
import re
from tqdm import trange
import argparse
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [185]:
from sklearn.preprocessing import LabelEncoder

In [186]:
with open('../LSTM/train_kr_all.pkl', 'rb') as f :
    train_kr = pickle.load(f)

In [187]:
train_kr['temp_code'] = train_kr.Script_num.map(lambda x : '-'.join(x.split('-')[:3]))

In [188]:
temp_code = 'UL-MV-01'
temp_script = []
temp_category = []
temp = ''
for i, row in train_kr.iterrows() :
    if temp_code == row.temp_code :
        temp += ' ' + ' '.join(row.Script)
    else :
        temp_code = row.temp_code
        temp_script.append(temp)
        temp_category.append(row.Category)
        temp = ''

In [189]:
temp_category = np.array(temp_category)

In [190]:
import collections

In [191]:
def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

In [290]:
vocabulary_size = 1500

In [291]:
data, count, dictionary, reverse_dictionary = build_dataset(' '.join(temp_script).split(' '))

In [292]:
from sklearn.feature_extraction.text import CountVectorizer

In [293]:
count_vectorizer = CountVectorizer(vocabulary = list(dictionary.keys()))

In [270]:
raw_data = pd.DataFrame(count_vectorizer.fit_transform(temp_script).toarray(), columns = list(dictionary.keys()))

In [271]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [272]:
cv = KFold(n_splits = 5, shuffle = True, random_state = 7)

In [273]:
gnb = GaussianNB()

In [274]:
for train_index, test_index in cv.split(raw_data) :
    gnb.fit(raw_data.iloc[train_index], temp_category[train_index])
    yhat = gnb.predict(raw_data.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.43      0.43      0.43        47
         선박       0.44      0.42      0.43        53
         양묘       0.08      0.56      0.14        16
         육상       0.56      0.30      0.39       133
         이동       0.14      0.22      0.17        49
         입항       0.27      0.33      0.30        54
         접안       0.26      0.30      0.28        53
         출항       0.42      0.24      0.30        92
         통과       0.38      0.20      0.26       100
         투묘       0.21      0.15      0.18        40
         횡단       0.30      0.48      0.37        23

avg / total       0.37      0.30      0.31       660

             precision    recall  f1-score   support

         도선       0.31      0.43      0.36        35
         선박       0.35      0.27      0.31        55
         양묘       0.04      0.19      0.07        21
         육상       0.41      0.31      0.35       125
         이동       0.23      0.39      0.29

In [275]:
bnb = BernoulliNB()

In [276]:
for train_index, test_index in cv.split(raw_data) :
    bnb.fit(raw_data.iloc[train_index], temp_category[train_index])
    yhat = bnb.predict(raw_data.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.86      0.64      0.73        47
         선박       0.86      0.70      0.77        53
         양묘       0.50      0.12      0.20        16
         육상       0.66      0.82      0.73       133
         이동       0.58      0.67      0.62        49
         입항       0.80      0.69      0.74        54
         접안       0.74      0.87      0.80        53
         출항       0.84      0.83      0.84        92
         통과       0.68      0.74      0.71       100
         투묘       0.62      0.57      0.60        40
         횡단       0.82      0.39      0.53        23

avg / total       0.73      0.72      0.72       660

             precision    recall  f1-score   support

         도선       0.62      0.57      0.60        35
         선박       0.86      0.56      0.68        55
         양묘       0.00      0.00      0.00        21
         육상       0.55      0.70      0.62       125
         이동       0.67      0.67      0.67

In [277]:
mnb = MultinomialNB()

In [278]:
for train_index, test_index in cv.split(raw_data) :
    mnb.fit(raw_data.iloc[train_index], temp_category[train_index])
    yhat = mnb.predict(raw_data.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.80      0.70      0.75        47
         선박       0.86      0.72      0.78        53
         양묘       0.40      0.12      0.19        16
         육상       0.72      0.76      0.74       133
         이동       0.54      0.69      0.61        49
         입항       0.75      0.74      0.75        54
         접안       0.76      0.77      0.77        53
         출항       0.79      0.83      0.81        92
         통과       0.70      0.71      0.70       100
         투묘       0.60      0.62      0.61        40
         횡단       0.75      0.65      0.70        23

avg / total       0.72      0.72      0.72       660

             precision    recall  f1-score   support

         도선       0.58      0.51      0.55        35
         선박       0.84      0.56      0.67        55
         양묘       0.75      0.14      0.24        21
         육상       0.58      0.72      0.64       125
         이동       0.59      0.67      0.63

### Using tf-idf

In [294]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [295]:
tf = TfidfVectorizer(vocabulary = list(dictionary.keys()))

In [296]:
raw_data2 = pd.DataFrame(tf.fit_transform(temp_script).toarray(), columns = list(dictionary.keys()))

In [297]:
import xgboost

In [300]:
xgbc = xgboost.XGBClassifier(max_depth = 5, n_jobs = 12, n_estimators = 500)

In [301]:
for train_index, test_index in cv.split(raw_data) :
    xgbc.fit(raw_data.iloc[train_index], temp_category[train_index])
    yhat = xgbc.predict(raw_data.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.76      0.74      0.75        47
         선박       0.78      0.81      0.80        53
         양묘       0.56      0.62      0.59        16
         육상       0.72      0.74      0.73       133
         이동       0.73      0.73      0.73        49
         입항       0.80      0.74      0.77        54
         접안       0.87      0.89      0.88        53
         출항       0.82      0.84      0.83        92
         통과       0.80      0.68      0.74       100
         투묘       0.70      0.75      0.72        40
         횡단       0.66      0.83      0.73        23

avg / total       0.76      0.76      0.76       660

             precision    recall  f1-score   support

         도선       0.63      0.77      0.69        35
         선박       0.80      0.71      0.75        55
         양묘       0.80      0.57      0.67        21
         육상       0.62      0.70      0.66       125
         이동       0.73      0.75      0.74

In [302]:
from sklearn.ensemble import RandomForestClassifier

In [312]:
rfc = RandomForestClassifier(max_depth = None, n_jobs = 12, n_estimators = 1000)

In [313]:
for train_index, test_index in cv.split(raw_data2) :
    rfc.fit(raw_data2.iloc[train_index], temp_category[train_index])
    yhat = rfc.predict(raw_data2.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.86      0.66      0.75        47
         선박       0.80      0.68      0.73        53
         양묘       0.71      0.31      0.43        16
         육상       0.65      0.88      0.75       133
         이동       0.75      0.67      0.71        49
         입항       0.86      0.70      0.78        54
         접안       0.81      0.81      0.81        53
         출항       0.80      0.85      0.83        92
         통과       0.77      0.77      0.77       100
         투묘       0.81      0.65      0.72        40
         횡단       0.73      0.70      0.71        23

avg / total       0.77      0.76      0.75       660

             precision    recall  f1-score   support

         도선       0.74      0.66      0.70        35
         선박       0.86      0.55      0.67        55
         양묘       0.50      0.24      0.32        21
         육상       0.48      0.78      0.60       125
         이동       0.67      0.61      0.64

In [309]:
from sklearn.linear_model import LogisticRegression

In [310]:
lrc = LogisticRegression(C = 0.8, max_iter = 500, n_jobs = 12, solver = 'sag')

In [311]:
for train_index, test_index in cv.split(raw_data2) :
    lrc.fit(raw_data2.iloc[train_index], temp_category[train_index])
    yhat = lrc.predict(raw_data2.iloc[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.82      0.66      0.73        47
         선박       0.88      0.70      0.78        53
         양묘       0.73      0.50      0.59        16
         육상       0.64      0.83      0.73       133
         이동       0.77      0.67      0.72        49
         입항       0.85      0.72      0.78        54
         접안       0.91      0.79      0.85        53
         출항       0.79      0.84      0.81        92
         통과       0.71      0.75      0.73       100
         투묘       0.67      0.65      0.66        40
         횡단       0.72      0.57      0.63        23

avg / total       0.76      0.75      0.75       660

             precision    recall  f1-score   support

         도선       0.75      0.60      0.67        35
         선박       0.84      0.58      0.69        55
         양묘       0.70      0.33      0.45        21
         육상       0.50      0.80      0.62       125
         이동       0.69      0.65      0.67

In [314]:
import lightgbm

In [358]:
lgbmc = lightgbm.LGBMClassifier(n_jobs = 12, n_estimators = 500, subsample = 0.7)

In [359]:
for train_index, test_index in cv.split(raw_data2) :
    lgbmc.fit(raw_data2.iloc[train_index].values, temp_category[train_index])
    yhat = lgbmc.predict(raw_data2.iloc[test_index].values)
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.84      0.68      0.75        47
         선박       0.73      0.81      0.77        53
         양묘       0.57      0.50      0.53        16
         육상       0.70      0.73      0.72       133
         이동       0.69      0.71      0.70        49
         입항       0.75      0.72      0.74        54
         접안       0.86      0.83      0.85        53
         출항       0.79      0.84      0.81        92
         통과       0.75      0.69      0.72       100
         투묘       0.69      0.72      0.71        40
         횡단       0.62      0.70      0.65        23

avg / total       0.74      0.74      0.74       660

             precision    recall  f1-score   support

         도선       0.63      0.63      0.63        35
         선박       0.78      0.78      0.78        55
         양묘       0.71      0.57      0.63        21
         육상       0.59      0.73      0.65       125
         이동       0.64      0.68      0.66

In [323]:
from sklearn.svm import LinearSVC

In [339]:
lsvc = LinearSVC(C = 0.7)

In [340]:
for train_index, test_index in cv.split(raw_data2) :
    lsvc.fit(raw_data2.iloc[train_index].values, temp_category[train_index])
    yhat = lsvc.predict(raw_data2.iloc[test_index].values)
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.76      0.68      0.72        47
         선박       0.85      0.83      0.84        53
         양묘       0.75      0.75      0.75        16
         육상       0.72      0.74      0.73       133
         이동       0.70      0.67      0.69        49
         입항       0.84      0.78      0.81        54
         접안       0.86      0.83      0.85        53
         출항       0.82      0.83      0.82        92
         통과       0.70      0.74      0.72       100
         투묘       0.67      0.72      0.70        40
         횡단       0.65      0.65      0.65        23

avg / total       0.76      0.76      0.76       660

             precision    recall  f1-score   support

         도선       0.71      0.71      0.71        35
         선박       0.77      0.75      0.76        55
         양묘       0.72      0.62      0.67        21
         육상       0.59      0.75      0.66       125
         이동       0.74      0.68      0.71

### Word Embedding으로 calssification 해보면?

In [132]:
from gensim.models import Word2Vec

In [133]:
embedding_model = Word2Vec.load('../LSTM/kr_word2vec_noun.model')

In [134]:
def get_mean_vector(sentence) :
    words = sentence.split()
    result = []
    for word in words :
        try :
            result.append(embedding_model.wv.get_vector(word))
        except :
            continue
    result = np.array(result)
    if result.sum() == 0 :
        result = np.zeros((1, 300))
    result = result.mean(axis = 0)
    return result
    

In [135]:
embedding_data = np.array(None)
for sentence in temp_script :
    embedding_data = np.append(embedding_data, get_mean_vector(sentence))

In [136]:
embedding_data = np.delete(embedding_data, 0)

In [137]:
embedding_data = embedding_data.reshape(-1, 300)

In [ ]:
xgbc = xgboost.XGBClassifier(max_depth = 3, n_jobs = 12, n_estimators = 500)

In [138]:
for train_index, test_index in cv.split(embedding_data) :
    xgbc.fit(embedding_data[train_index], temp_category[train_index])
    yhat = xgbc.predict(embedding_data[test_index])
    print(classification_report(temp_category[test_index], yhat))

             precision    recall  f1-score   support

         도선       0.76      0.62      0.68        21
         선박       0.45      0.38      0.42        26
         양묘       0.67      0.36      0.47        11
         육상       0.61      0.75      0.67        63
         이동       0.46      0.52      0.49        21
         입항       0.85      0.65      0.74        26
         접안       0.82      0.78      0.80        23
         출항       0.79      0.85      0.81        52
         통과       0.59      0.67      0.63        52
         투묘       0.88      0.71      0.79        21
         횡단       0.60      0.43      0.50        14

avg / total       0.68      0.67      0.66       330

             precision    recall  f1-score   support

         도선       0.81      0.50      0.62        26
         선박       0.70      0.59      0.64        27
         양묘       0.50      0.40      0.44         5
         육상       0.63      0.80      0.70        70
         이동       0.86      0.64      0.73

In [ ]:
lrc = LogisticRegression(C = 0.8, max_iter = 500, n_jobs = 12, solver = 'lbfgs')

In [ ]:
for train_index, test_index in cv.split(embedding_data) :
    lrc.fit(embedding_data[train_index], temp_category[train_index])
    yhat = lrc.predict(embedding_data[test_index])
    print(classification_report(temp_category[test_index], yhat))

### Time information 반영

In [350]:
temp_code = 'UL-MV-01'
temp_script2 = []
temp_category2 = []
temp2 = ''
step = []
temp_step = 0
for i, row in train_kr.iterrows() :
    temp_step += 1
    if temp_code == row.temp_code :
        temp2 += ' ' + ' '.join(row.Script)
        temp_script2.append(temp2)
        temp_category2.append(row.Category)
        step.append(temp_step)
    else :
        temp_code = row.temp_code
        temp_script2.append(temp2)
        temp_category2.append(row.Category)
        temp2 = ''
        step.append(temp_step)
        temp_step = 0

In [351]:
temp_df = pd.DataFrame(tf.transform(temp_script2).toarray(), columns = list(dictionary.keys()))

In [352]:
temp_df['category'] = temp_category2
temp_df['step'] = step

In [353]:
temp_df.step.max()

25

상위 3개 예측 category에 실제 label이 들어가면 정답

In [354]:
for i in range(1, 25) :
    data = temp_df[temp_df.step == i].drop(['category', 'step'], axis = 1)
    yhat = xgbc.predict(data)
    print(classification_report(temp_df[temp_df.step == i].category.values, yhat), i)

ValueError: feature_names mismatch: ['', '수고', '현장', '못', '속도', '대단히', '브라보', '방', '되', '달포', '인', '리지', '최대한', '히', '정정', '여', '위스키', '비키', '직각', '우리', '혀', '공업', 'vts', '랍니', '파일럿', '전부', '세요', '낙', '사포', '마', '이에', '바짝', '불르', '대산', '에', '또', '진해', '침해', '꺼', '고맙', '부터', '찰', '행', '보드', '많', '사천', '데로', '다르', '앞쪽', '콜사인', '안전점검', '접안', '벽', '인원', '총', '잘', 'n', '바', '돌', '지나', '광', '중인', '풀', '부', '당직', '미로', 'calling', '반', 'lng', '함대', '호출', '서쪽', '통과', '평택갑니', '석', '으로', '빠르', '날씨', '움직이', '사이드', '잡', '어디', '하면', '전선', '목적지', '바로', '상', '이랑', 'reporting', 'dsnr', '들어오', '을', '단', 'ㄴ다고', '던지', '곳', '치고', 'cps', '서로', '일', '오르', 'ps', ',', '다가', 'morning', '크레인', '올라오', '일반부', '만', 'dsf', '많이', '하역', '갈라', '아침', '붙', '택', '고려', '함', '수면', '워터', '해군', '참조', '수가', '감사', '승무원', '내', '현재', 'jk', '이러', '안받', '관제', '러', '더', '신속', '유지', '도선', '길', '소형', 'ㄹ라', '교', '물어보', '대우', '석탄', '뒤따르', 'cpa', '지점', '들리', '양키', '마일', '북측', '지금', '빠지', '채널', 'to', '승선', '잇', 'ㄴ데', '오산', '교대', '넘어가', '저